In [104]:
import sys, os
from collections import deque

import h5py

from aliases import *

In [108]:
osjoin = os.path.join
def get_framepaths(datadir):
    frmPaths = []
    subDirs = [
        p for p in (osjoin(datadir, n) for n in os.listdir(datadir))
            if os.path.isdir(p)
        ]
    for subDir in subDirs:
        frmPaths.extend(
            osjoin(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
            )
    return frmPaths

In [110]:
os.path.basename('/home/morpheus/workspace/mount/data/charon5/linear2.frm')

'linear2.frm'

In [112]:
osrel = os.path.relpath

In [113]:
osrel('/home/morpheus/workspace/mount/data/charon5/linear2.frm', datadir)

'charon5/linear2.frm'

In [109]:
get_framepaths(datadir).index('/home/morpheus/workspace/mount/data/pluto1/.frm')

['/home/morpheus/workspace/mount/data/charon5/linear2.frm',
 '/home/morpheus/workspace/mount/data/miranda1/plasticaspectfocus.frm',
 '/home/morpheus/workspace/mount/data/miranda1/linear2.frm',
 '/home/morpheus/workspace/mount/data/miranda1/plasticaspectfocus2.frm',
 '/home/morpheus/workspace/mount/data/miranda1/plasticfull.frm',
 '/home/morpheus/workspace/mount/data/miranda1/plasticaspect3.frm',
 '/home/morpheus/workspace/mount/data/miranda1/plasticannu.frm',
 '/home/morpheus/workspace/mount/data/miranda1/miranda1.frm',
 '/home/morpheus/workspace/mount/data/triton/MS98.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/linear.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/linear2.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/plasticheat.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/plasticfull.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/plasticfull2.frm',
 '/home/morpheus/workspace/mount/data/umbriel3/plasticannu.frm',
 '/home/morpheus/workspace/mount/dat

In [105]:
with h5py.File(os.path.join(datadir, 'pluto1', 'linear2.frm'), mode = 'r') as h5file:
    print(h5file.keys())

RuntimeError: Link iteration failed (bad heap free list)

In [55]:
frmPaths = []
subDirs = [
    p for p in (os.path.join(datadir, n) for n in os.listdir(datadir))
        if os.path.isdir(p)
    ]
for subDir in subDirs:
    frmPaths.extend(
        os.path.join(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
        )

In [69]:
h5py.Group.copy

<function h5py._hl.group.Group.copy(self, source, dest, name=None, shallow=False, expand_soft=False, expand_external=False, expand_refs=False, without_attrs=False)>

In [70]:
h5py.ExternalLink

h5py._hl.group.ExternalLink

In [76]:
'_string_foo'[8:]

'foo'

In [102]:
%%time

observees = dict()

frmPath = frmPaths[0]
with h5py.File('temp.frm', mode = 'w') as writefile:
    with h5py.File(frmPath, mode = 'r') as readfile:
        for i, (key, grp) in enumerate(readfile.items()):
            if not i % 1000:
                print('.')
            try:
                styp = grp.attrs['supertype'][8:]
                if styp == 'System':
                    wgrp = writefile.require_group(key)
                    wgrp.attrs.update(grp.attrs)
                    wgrp['inputs'] = h5py.ExternalLink(frmPath, os.path.join(key, 'inputs'))
                elif styp == 'Observer':
                    typ = grp.attrs['type'][8:]
                    wgrp = writefile.require_group(grp['inputs'].attrs['observee'][7:])
                    okey = typ.lower()
                    if okey in wgrp:
                        if len(wgrp[okey]['count']) > len(grp['outputs']['count']):
                            continue
                    wgrp[okey] = h5py.ExternalLink(frmPath, os.path.join(key, 'outputs'))
            except (KeyError, RuntimeError):
                continue

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
CPU times: user 8.33 s, sys: 433 ms, total: 8.76 s
Wall time: 8.76 s


In [90]:
myfile = h5py.File('temp.frm', mode = 'r')

In [ ]:
myfile['aabrspuugh-eistiopoov']

In [91]:
myfile.close()

In [103]:
%%time
with h5py.File('temp.frm', mode = 'r') as h5file:
    print(len(h5file))
    out = h5file['aabrspuugh-eistiopoov']['thermo']['Nu'][()]

2917
CPU times: user 4.93 ms, sys: 0 ns, total: 4.93 ms
Wall time: 5.03 ms


In [ ]:
# if not __name__ == '__main__':
#     raise RuntimeError

# import os
# from collections import deque

# import h5py

# from aliases import *

# frmPaths = []
# subDirs = [
#     p for p in (os.path.join(datadir, n) for n in os.listdir(datadir))
#         if os.path.isdir(p)
#     ]
# for subDir in subDirs:
#     frmPaths.extend(
#         os.path.join(subDir, n) for n in os.listdir(subDir) if (n.endswith('.frm'))
#         )

# with h5py.File(os.path.join(datadir, 'allout.frm'), mode = 'w') as writefile:
#     for frmPath in frmPaths:
#         print('-' * 10 + frmPath + '-' * 10)
#         with h5py.File(frmPath, mode = 'r') as readfile:
#             for i, (key, grp) in enumerate(readfile.items()):
#                 grpkeys = grp.keys()
#                 try:
#                     if not grp.attrs['supertype'] == '_string_Observer':
#                         continue
#                 except KeyError:
#                     continue
#                 if not i % 1000:
#                     print(key)
#                 try:
#                     readout = grp['outputs']
#                 except KeyError:
#                     continue
#                 hashID = grp.attrs['hashID'][8:]
#                 writegrp = writefile.require_group(hashID)
#                 systemgrp = readfile[hashID]
#                 writegrp.attrs.update(systemgrp.attrs)
#                 try:
#                     writefile.copy(systemgrp['inputs'], writegrp)
#                 except RuntimeError:
#                     pass
#                 outgrp = writegrp.require_group('outputs')
#                 for dname, dset in readout.items():
#                     if dname in outgrp:
#                         continue
#                     if not dset:
#                         continue
#                     try:
#                         oset = outgrp.create_dataset(
#                             dname,
#                             data = dset[()],
#                             chunks = dset.shape
#                             )
#                     except OSError:
#                         try:
#                             oset = outgrp.create_dataset(
#                                 dname,
#                                 data = (sub := dset[:-10]),
#                                 chunks = sub.shape
#                                 )
#                         except OSError:
#                             continue

In [ ]:
#         for i, (key, grp) in enumerate(readfile.items()):
#             if not i % 1000:
#                 print('.')
#             try:
#                 observees[key] = grp['inputs'].attrs['observee'][7:]
#             except KeyError:
#                 continue

In [ ]:
def visit(key, grp):
    try:
        if not grp.attrs['supertype'] == '_string_Observer':
            continue
    except KeyError:
        return
    try:
        systemid = grp['inputs']['observee'][7:]
    except KeyError:
        return
    systemid

In [ ]:
    for i, (key, grp) in enumerate(readfile.items()):
        grpkeys = grp.keys()
        try:
            if not grp.attrs['supertype'] == '_string_Observer':
                continue
        except KeyError:
            continue
        if not i % 1000:
            print(key)
        try:
            readout = grp['outputs']
        except KeyError:
            continue
        hashID = grp.attrs['hashID'][8:]
        writegrp = writefile.require_group(hashID)
        systemgrp = readfile[hashID]
        writegrp.attrs.update(systemgrp.attrs)
        try:
            writefile.copy(systemgrp['inputs'], writegrp)
        except RuntimeError:
            pass
        outgrp = writegrp.require_group('outputs')
        for dname, dset in readout.items():
            if dname in outgrp:
                continue
            if not dset:
                continue
            try:
                oset = outgrp.create_dataset(
                    dname,
                    data = dset[()],
                    chunks = dset.shape
                    )
            except OSError:
                try:
                    oset = outgrp.create_dataset(
                        dname,
                        data = (sub := dset[:-10]),
                        chunks = sub.shape
                        )
                except OSError:
                    continue

In [47]:
frmPaths[0]

'/home/morpheus/workspace/mount/data/charon5/linear2.frm'

In [50]:
with h5py.File(frmPaths[0], mode = 'r') as h5file:
    print(list(h5file['xoosftzii-skorhgleuv'].items()))

[('inputs', <HDF5 group "/xoosftzii-skorhgleuv/inputs" (0 members)>), ('observee', <HDF5 group "/xoosftzii-skorhgleuv/observee" (6 members)>)]


In [3]:
testpath = frmPaths[0]

In [51]:
%%timeit
bool(1 % 1000)

67.7 ns ± 0.0456 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [32]:
help(h5py.Dataset.write_direct)

Help on function write_direct in module h5py._hl.dataset:

write_direct(self, source, source_sel=None, dest_sel=None)
    Write data directly to HDF5 from a NumPy array.
    
    The source array must be C-contiguous.  Selections must be
    the output of numpy.s_[<args>].
    
    Broadcasting is supported for simple indexing.



In [45]:
help(h5py.Group.create_dataset)

Help on function create_dataset in module h5py._hl.group:

create_dataset(self, name, shape=None, dtype=None, data=None, **kwds)
    Create a new HDF5 dataset
    
    name
        Name of the dataset (absolute or relative).  Provide None to make
        an anonymous dataset.
    shape
        Dataset shape.  Use "()" for scalar datasets.  Required if "data"
        isn't provided.
    dtype
        Numpy dtype or string.  If omitted, dtype('f') will be used.
        Required if "data" isn't provided; otherwise, overrides data
        array's dtype.
    data
        Provide data to initialize the dataset.  If used, you can omit
        shape and dtype arguments.
    
    Keyword-only arguments:
    
    chunks
        (Tuple or int) Chunk shape, or True to enable auto-chunking. Integers can
        be used for 1D shape.
    
    maxshape
        (Tuple or int) Make the dataset resizable up to this shape. Use None for
        axes you want to be unlimited. Integers can be used for 1D sh

In [31]:
help(h5py.Dataset.read_direct)

Help on function read_direct in module h5py._hl.dataset:

read_direct(self, dest, source_sel=None, dest_sel=None)
    Read data directly from HDF5 into an existing NumPy array.
    
    The destination array must be C-contiguous and writable.
    Selections must be the output of numpy.s_[<args>].
    
    Broadcasting is supported for simple indexing.



In [40]:
help(h5py.Group.copy)

Help on function copy in module h5py._hl.group:

copy(self, source, dest, name=None, shallow=False, expand_soft=False, expand_external=False, expand_refs=False, without_attrs=False)
    Copy an object or group.
    
     The source can be a path, Group, Dataset, or Datatype object.  The
     destination can be either a path or a Group object.  The source and
     destinations need not be in the same file.
    
     If the source is a Group object, all objects contained in that group
     will be copied recursively.
    
     When the destination is a Group object, by default the target will
     be created in that group with its current name (basename of obj.name).
     You can override that by setting "name" to a string.
    
     There are various options which all default to "False":
    
      - shallow: copy only immediate members of a group.
    
      - expand_soft: expand soft links into new objects.
    
      - expand_external: expand external links into new objects.
    
   

In [38]:
ah5py.File.copy()

TypeError: copy() missing 3 required positional arguments: 'self', 'source', and 'dest'

In [41]:
h5py.copy

AttributeError: module 'h5py' has no attribute 'copy'

In [44]:
with h5py.File('temp.frm', mode = 'w') as writefile:
    with h5py.File(os.path.join(testpath), mode = 'r') as readfile:
        for i, (key, grp) in enumerate(readfile.items()):
            if i > 1000:
                print('.')
            grpkeys = grp.keys()
            if not 'observee' in grpkeys:
                continue
            hashID = grp.attrs['hashID'][8:]
            writegrp = writefile.require_group(hashID)
            writefile.copy(readfile[hashID]['inputs'], writegrp)
            readfile.copy(grp['outputs'], writegrp, name = grp.attrs['type'])

aabeapleozh-xeewrspueph
aabloieoblua-kronbriung
aabruoqoosp-vaodwuezhio
aacaewapr-eikaaeaswoo
aachianeakw-iugsfaopl
aachoeheufl-hiiprsnoifl
aacoeceath-klaitrskui
aaflaughoth-kweoytheiph
aafloaweef-ngourhgluakr
aafoauofa-priokwich
aagaispoekl-kluosfsmoodr
aaghuuchoatz-eiglaismeosh
aaguokiis-iusfeeatsei
aaheiuozhea-iuhuadraath
aahooisnia-dwuayweedw
aajasfiisn-droiroupreo
aakouwiing-iegloachaoz
aakwoirhui-aefaosoikr
aaliapsou-spaiphkoub
aamiograo-uitruogreig
aaphaaipa-freuchrhaesk
aaphoezhaiq-xautzgrac
aapleakeogl-stuazhfreudr
aapluaepsoi-ooslaitsain
aapseithoej-skeigrmoaz
aaqublioth-iawraocad
aarhaabrim-dooblzoekr
aarheegliesm-spoatghausp
aarhoosael-snouwruobei
aascauaoglio-aikeuuigoe
aaseaeyeu-blaouichuo
aasfeoaakroe-uifroiatria
aashagretw-smaofiaraa
aasieathai-matsxuatz
aaskeonou-fruashtruukr
aaskuedreoq-ngoosrood
aasliouetwiu-huovoaskuo
aasmaaaigu-leeshciudr
aasmaouujao-phedrrhaodw
aaspeostu-xeigloekwie
aastatsau-waedoazuo
aastoeghiph-euhaghoe
aasuijiap-euploabeetw
aasweulaokw-tsoekud

KeyboardInterrupt: 

In [18]:
len(keys)

20023

In [ ]:
with h5py.Fil(os.path.join(datadir, 'out.frm'), mode = 'r') as writefile:
    with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
        

In [ ]:
reader = Reader('mergedcopy', datadir)

In [ ]:
i = 0
out = []
def visitprint(name):
    global i
    global out
    i += 1
    if i > 100:
        return True
    out.append(name)

In [ ]:
out = []
with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
#     readfile.visit(visitprint)
    readfile.visit(out.append)

In [ ]:
len(out)

In [ ]:
links = []
with h5py.File(os.path.join(datadir, 'mergedcopy.frm'), mode = 'r') as readfile:
    readfile.id.links.iterate(links.append)

In [ ]:
datasets = dict()
with h5py.File(os.path.join(datadir, 'merged.frm'), mode = 'r') as readfile:
    links = deque()
    readfile.id.links.iterate(links.append)
    subpaths = deque()
    for i, link in enumerate(links):
        if not i % 1000:
            print('.')
        subpaths.clear()
        readfile[link].visit(subpaths.append)
        if 'observee' in subpaths:
            target = datasets.setdefault(
                readfile[link]['observee'].attrs['hashID'][8:],
                deque()
                )
            linkmaker = lambda x: target.append(os.path.join(link.decode(), 'outputs', x))
            readfile[link]['outputs'].visit(linkmaker)